In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [ ]:
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

In [ ]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
#X_train = train.copy()
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
del train,test,train_transaction,test_transaction,train_identity,test_identity

In [ ]:
y_value_counts = y_train.value_counts()
print(y_value_counts)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize = (6,6), subplot_kw = dict(aspect = 'equal'))
recipe = ["isFraud","isNotFraud"]
data = [y_value_counts[1], y_value_counts[0]]
print("Percentage of Transactions that are Fraudulent = ", y_value_counts[1]/(y_value_counts[1]+y_value_counts[0]), " %")

data = [y_value_counts[1], y_value_counts[0]]

wedges, texts = ax.pie(data, wedgeprops=dict(width=0.5), startangle=40)

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(xycoords='data', textcoords='data', arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(recipe[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                 horizontalalignment=horizontalalignment, **kw)

ax.set_title("Number of Fraudulent transactions and Non-Fraudulent transactions")
plt.show()
    

Summary: The number of non-fraudulent transactions is very high compared to the number of non fraudulent transactions. The dataset is imbalanced.

In [ ]:
a = []
for i in X_train.columns:
    a.append(X_train[i].dtype)
pd.Series(a).value_counts()

In [ ]:
'''project_data = X_train
for i in X_train.columns:
    if X_train[i].dtype=='object':
        print('='*50)
        print('For column: ',i)
        col1 = i
        col2 = 'isFraud'
        col3 = 'total'
        
        # Count number of zeros in dataframe python: https://stackoverflow.com/a/51540521/4084039
        temp = pd.DataFrame(project_data.groupby(col1)[col2].agg(lambda x: x.eq(1).sum())).reset_index()

        # Pandas dataframe grouby count: https://stackoverflow.com/a/19385591/4084039
        temp['total'] = pd.DataFrame(project_data.groupby(col1)[col2].agg({'total':'count'})).reset_index()['total']
        temp['Avg'] = pd.DataFrame(project_data.groupby(col1)[col2].agg({'Avg':'mean'})).reset_index()['Avg']

        temp.sort_values(by=['total'],inplace=True, ascending=False)

        data = temp
        ind = np.arange(data.shape[0])
        
        plt.figure(figsize=(10,5))
        p1 = plt.bar(ind, data[col3].values, width = 0.35)
        p2 = plt.bar(ind, data[col2].values, width = 0.35)

        plt.ylabel('Projects')
        plt.title('Number of transactions fraudulent vs not-fraudulent')
        plt.xticks(ind, list(data[col1].values))
        plt.legend((p1[0], p2[0]), ('total', 'Fraudulent'))
        plt.show()


        print(temp.head(5))
        print("="*50)
        print(temp.tail(5))'''

In [ ]:
'''import seaborn as sns'''

In [ ]:
'''cnt = 0
for i in X_train.columns:
    if cnt<50:
        cnt+=1
        if X_train[i].dtype=='Float64':
            print('='*50)
            print('For column: ',i)
            isFraud = project_data[project_data['isFraud']==1][i].values
            notFraud = project_data[project_data['isFraud']==0][i].values

            # https://glowingpython.blogspot.com/2012/09/boxplot-with-matplotlib.html
            plt.boxplot([isFraud,notFraud])
            plt.title('Box Plot:')
            plt.xticks([1,2],('isFraud','notFraud'))
            plt.ylabel(X_train.columns[1])
            plt.grid()
            plt.show()


            plt.figure(figsize=(10,3))
            sns.distplot(isFraud, hist=False, label="isFraud")
            sns.distplot(notFraud, hist=False, label="notFraud")
            plt.title('Distribution Plot')
            plt.xlabel(i)
            plt.legend()
            plt.show()

            k = -999
            if k in X_train[i].values:
                print('percentage of missing values:', 100*(X_train[i].value_counts()[-999])/len(X_train))
            else:
                print('There are no missing values in this feature')
        '''

In [ ]:
from sklearn import preprocessing
# Label Encoding
for f in X_train.columns:
    if (X_train[f].dtype=='object'): 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values)+list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))

Simple NaiveBayes

In [ ]:
'''# Ref1 https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist
# Ref2 https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/
#X_train = X_train.drop('isFraud', axis=1)
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
input_shape = (X_train.shape[0], X_train.shape[1])
model = Sequential()
model.add(Conv1D(64, kernel_size=5,activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(32, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(np.array(X_train), y_train,
          epochs=5,
          verbose=1)
#score = model.evaluate(X_train, y_train, verbose=0)
#print('Train loss:', score[0])
#print('Train accuracy:', score[1])'''

In [ ]:
'''clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999)

clf.fit(X_train, y_train)'''

In [ ]:
'''sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('simple_xgboost.csv')'''

In [ ]:
print(X_train.shape, X_test.shape, len(y_train))

In [ ]:
from datetime import datetime
start = datetime.now()
'''from sklearn.naive_bayes import MultinomialNB as MN
clf = MN(class_prior=[0.5,0.5])
print('Model Defined')
clf.fit(abs(X_train),y_train)
print('Fit Done!, Time taken: ', datetime.now() - start)
predate_1 = clf.predict_proba(abs(X_test))[:,1]'''
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(max_depth = 10, class_weight = 'balanced')
print('Model Defined')
clf.fit(X_train,y_train)
predate_2 = clf.predict_proba(X_test)[:,1]
print('Fit Done!, Time taken: ', datetime.now() - start)
print(clf.feature_importances_)

In [ ]:
#predate = (0.8169*predate_1+0.6*predate_2)/(0.8169+0.6)
predate = predate_2
'''for i in range(len(predate)):
    if predate[i]>=0.5:
        predate[i]=1
    else:
        predate[i]=0'''
sample_submission['isFraud'] = predate
sample_submission.to_csv('weighted_NB_RF.csv')